In [4]:
%load_ext autoreload
%autoreload 2
# prepend parent dir to import from local pcsp
import sys

sys.path.insert(0, '../')

from vflow import Vset, init_args
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from functools import partial

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# load data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [8]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))


# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, fc1=512, fc2=512):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, fc1),
            nn.ReLU(),
            nn.Linear(fc1, fc2),
            nn.ReLU(),
            nn.Linear(fc2, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


models = [NeuralNetwork(), NeuralNetwork(256, 512)]
loss_fn = nn.CrossEntropyLoss()

Using cpu device


In [12]:
def train(dataloader, model, loss_fn, epochs):
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    size = len(dataloader.dataset)
    for t in range(epochs):
        # print(f"Epoch {t+1}\n-------------------------------")
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # Compute prediction error
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 100 == 0:
                loss, current = loss.item(), batch * len(X)
                # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return model


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    return correct

In [13]:
train_data, test_data = init_args((train_dataloader, test_dataloader), names=['train_data', 'test_data'])

# fit neural network
train_nn = [partial(train, model=model, loss_fn=loss_fn, epochs=5) for model in models]
modeling_set = Vset(name='modeling', modules=train_nn)
modeling_set.fit(train_data)

In [14]:
test_nn = partial(test, loss_fn=loss_fn)
hard_metrics_set = Vset(name='hard_metrics', modules=[test_nn])
hard_metrics_set.evaluate(test_data, modeling_set.out)

{(test_data, train_data, modeling_0, hard_metrics_0): 0.4344,
 (test_data, train_data, modeling_1, hard_metrics_0): 0.4259,
 '__prev__': (<vflow.vset.Vset at 0x7fdec559a470>,)}